# Based on code from https://www.kaggle.com/code/muhammadfaizan65/machine-failure-prediction-eda-modeling. Added new models and comparison. Data can be found here https://www.kaggle.com/datasets/umerrtx/machine-failure-prediction-using-sensor-data?resource=download. 

# Dataset Overview
This dataset contains sensor data collected from various machines, to predict machine failures in advance. It includes a variety of sensor readings as well as recorded machine failures.

# Columns Description
footfall: The number of people or objects passing by the machine.  
tempMode: The temperature mode or setting of the machine.  
AQ: Air quality index near the machine.  
USS: Ultrasonic sensor data, indicating proximity measurements.  
CS: Current sensor readings, indicating the electrical current usage of the machine.  
VOC: Volatile organic compounds level detected near the machine.  
RP: Rotational position or RPM (revolutions per minute) of the machine parts.  
IP: Input pressure to the machine.  
Temperature: The operating temperature of the machine.  
fail: Binary indicator of machine failure (1 for failure, 0 for no failure).

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score
import joblib

In [ ]:
# Load the dataset
file_path = "data.csv"
data = pd.read_csv(file_path)

In [ ]:
# Display basic info and summary
print(data.info())
print(data.describe())
print(data.shape)

In [ ]:
# Check for missing values
print(data.isnull().sum())

In [ ]:
# Distribution of numeric columns
fig = make_subplots(rows=5, cols=2, subplot_titles=data.columns)
for i, column in enumerate(data.columns):
    row = i // 2 + 1
    col = i % 2 + 1
    hist = px.histogram(data, x=column, template='plotly_dark', color_discrete_sequence=['#F63366'])
    hist.update_traces(marker_line_width=0.5, marker_line_color="white")
    fig.add_trace(hist.data[0], row=row, col=col)

fig.update_layout(height=1200, title_text="Distribution of Numeric Columns", title_font=dict(size=25), title_x=0.5, showlegend=False)
fig.show()

In [ ]:

# Correlation Heatmap
corr = data.corr()
fig = ff.create_annotated_heatmap(
    z=corr.values,
    x=list(corr.columns),
    y=list(corr.index),
    annotation_text=corr.round(2).values,
    showscale=True,
    colorscale='Viridis')
fig.update_layout(title_text='Correlation Heatmap', title_font=dict(size=25), title_x=0.5)
fig.show()

In [ ]:
# Boxplots for each feature to identify outliers
fig = make_subplots(rows=5, cols=2, subplot_titles=data.columns[:-1])
for i, column in enumerate(data.columns[:-1]):  # Excluding the target column 'fail'
    row = i // 2 + 1
    col = i % 2 + 1
    box = px.box(data, y=column, template='plotly_dark', color_discrete_sequence=['#636EFA'])
    box.update_traces(marker_line_width=0.5, marker_line_color="white")
    fig.add_trace(box.data[0], row=row, col=col)

fig.update_layout(height=1200, title_text="Boxplots of Features", title_font=dict(size=25), title_x=0.5, showlegend=False)
fig.show()

In [ ]:
# Scatter plots to visualize relationships between features and target
fig = make_subplots(rows=5, cols=2, subplot_titles=data.columns[:-1])
for i, column in enumerate(data.columns[:-1]):  # Excluding the target column 'fail'
    row = i // 2 + 1
    col = i % 2 + 1
    scatter = px.scatter(data, x=column, y='fail', template='plotly_dark', color='fail', color_continuous_scale='Viridis')
    scatter.update_traces(marker=dict(size=5, opacity=0.7, line=dict(width=0.5, color='white')))
    fig.add_trace(scatter.data[0], row=row, col=col)

fig.update_layout(height=1200, title_text="Scatter Plots of Features vs Fail", title_font=dict(size=25), title_x=0.5, showlegend=False)
fig.show()

In [ ]:
# Data Preprocessing
X = data.drop(columns=['fail'])
y = data['fail']

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Model Training and Hyperparameter Tuning
models = {
    'RandomForest': RandomForestClassifier(),
    'GradientBoosting': GradientBoostingClassifier(),
    'LogisticRegression': LogisticRegression(),
    'SVM': SVC(probability=True)
}

params = {
    'RandomForest': {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]},
    'GradientBoosting': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 1]},
    'LogisticRegression': {'C': [0.01, 0.1, 1, 10]},
    'SVM': {'C': [0.01, 0.1, 1, 10], 'kernel': ['linear', 'rbf']}
}

In [ ]:
best_models = {}
best_accuracy = 0
best_model_name = ''
best_model = None

for model_name in models.keys():
    grid = GridSearchCV(models[model_name], params[model_name], cv=5, n_jobs=-1, verbose=1)
    grid.fit(X_train, y_train)
    best_models[model_name] = grid.best_estimator_
    print(f"Best parameters for {model_name}: {grid.best_params_}")
    y_pred = grid.best_estimator_.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for {model_name}: {accuracy}")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model_name = model_name
        best_model = grid.best_estimator_

print(f"\nBest model: {best_model_name} with accuracy: {best_accuracy}")

In [ ]:
# Evaluation of the best model
y_pred = best_model.predict(X_test)
y_prob = best_model.predict_proba(X_test)[:, 1]
report = classification_report(y_test, y_pred, output_dict=True)
cm = confusion_matrix(y_test, y_pred)

In [ ]:

# Plot confusion matrix
fig = px.imshow(cm, text_auto=True, color_continuous_scale='Blues', template='plotly_dark')
fig.update_layout(title_text=f'Confusion Matrix for {best_model_name}', title_font=dict(size=25), title_x=0.5)
fig.show()

In [ ]:
# Print classification report
report_df = pd.DataFrame(report).transpose()
print(f'Classification Report for {best_model_name}')
print(report_df)

In [ ]:
# ROC Curve and AUC for the best model
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'{best_model_name} (AUC = {roc_auc:.2f})', line=dict(width=2)))
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line=dict(dash='dash', color='gray'), name='Random'))
fig.update_layout(title_text='Receiver Operating Characteristic', title_font=dict(size=25), xaxis_title='False Positive Rate', yaxis_title='True Positive Rate', template='plotly_dark')
fig.show()

# Receiver Operating Characteristic (ROC) Curve

## Interpretation

The ROC curve displayed above shows the performance of the **RandomForest** classifier on the test dataset. The curve plots the True Positive Rate (TPR) against the False Positive Rate (FPR) at various threshold settings.

### Key Points:

- **True Positive Rate (TPR)**: Also known as Sensitivity or Recall, it is the ratio of correctly predicted positive observations to the actual positives.
- **False Positive Rate (FPR)**: It is the ratio of incorrectly predicted positive observations to the actual negatives.

### Analysis:

- A perfect classifier would have an AUC of **1.0**, while a classifier with no discriminative power would have an AUC of **0.5** (represented by the dashed line labeled "Random").
- The ROC curve is very close to the top left corner, demonstrating that the model has a high TPR and a low FPR, meaning it correctly identifies a large proportion of positive cases while keeping false positives to a minimum.